<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/DataModelAntoineEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from lxml import html
import requests
import jax
import jax.numpy as jnp
import numpy as np
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import deque
pio.templates.default='plotly_dark'

In [4]:
original_url='http://www.ddbst.com/en/EED/PCP/VAP_C174.php'

url = 'https://raw.githubusercontent.com/profteachkids/CHE2064/master/data/DDBWaterVaporPressure.html'

content= requests.get(url).content
tree=html.fromstring(content)

In [5]:
T_strings=tree.xpath('//*[@id="online-ddb-form"]/div[2]/table[3]/tbody/tr/td[1]/text()')
P_strings=tree.xpath('//*[@id="online-ddb-form"]/div[2]/table[3]/tbody/tr/td[2]/text()')

In [6]:
data = np.array([T_strings, P_strings]).astype(np.float64)

In [12]:
T=data[0,:]
log_Pv = np.log10(1000*data[1,:])

In [13]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=T, y=log_Pv, mode='markers', marker_size=8, marker_color='rgba(0,0,0,0)',
                marker_line_color='rgb(0,0,255)', marker_line_width=1, name='Data')
fig.update_layout(width=800, height=500)
fig.update_layout(xaxis_title='T (K)', yaxis_title='$Log_{10} P\ (Pa)$a')
fig.show()

In [14]:
def model(a, b, c, T):
  return a-b/(T+c)

In [15]:
a_guess = 10
b_guess = 2000.0
c_guess = 0.0
x0 = [a_guess, b_guess, c_guess]

def sqerror(x, T, log_Pv):
  model_Pv = model(x[0],x[1],x[2], T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

def sqerror_scaled(x, T, log_Pv):
  model_Pv = model(x[0],x[1]*1000,x[2]*100, T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10)
# res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, method='COBYLA')
res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, jac=jax.grad(sqerror))

/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning:

No GPU/TPU found, falling back to CPU.



In [16]:
res

      fun: 0.000538109801709652
 hess_inv: array([[6.24618639e+00, 4.86215691e+03, 4.97409681e+02],
       [4.86215691e+03, 3.82909205e+06, 3.95747473e+05],
       [4.97409681e+02, 3.95747473e+05, 4.13633226e+04]])
      jac: DeviceArray([-1.8596649e-05,  4.3188265e-08, -1.9003710e-07], dtype=float32)
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 116
      nit: 39
     njev: 104
   status: 2
  success: False
        x: array([  10.20869152, 1756.35666244,  -36.02766077])

In [17]:
T_model = jnp.linspace(T[0],T[-1],100)
fig.add_scatter(x=T_model, y=model(*res.x, T_model), mode='lines', line_width=1, name='Antoine')
fig.show()